## 🥁 Setup

Change the following variable settings to match your deployed model's *Inference endpoint*. for example: 

```
deployed_model_name = "jukebox"
infer_endpoint = "https://jukebox-yyyyyy.apps.cluster-p9k5m.p9k5m.sandboxxxx.opentlc.com"
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import numpy as np
import requests

In [ ]:
deployed_model_name = "jukebox"
infer_endpoint = "<paste-the-link-here>"
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

    NOTE: Make sure that you change the infer_endpoint with your own model inference endpoint deployed in the earlier step.

## 🫡 Request Function

Build and submit the REST request. 

In [ ]:
def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "dense_input",
                "shape": [1, 13],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=False)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']


In [ ]:
#Load the scaler and label encoder
with open('models/jukebox/1/artifacts/scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)
    
with open('models/jukebox/1/artifacts/label_encoder.pkl', 'rb') as handle:
    label_encoder = pickle.load(handle)

In [ ]:
song_properties = pd.read_parquet('../99-data_prep/song_properties.parquet')
favorite_song = song_properties.loc[song_properties["name"]=="Not Like Us"]
favorite_song

In [ ]:
data = favorite_song[['is_explicit', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
prediction = rest_request(scaler.transform(data.values).tolist()[0])

#### Let's visualize the chance that Kendrick Lamar would be popular in the countries!

In [ ]:
plt.figure(figsize = (6, 8))
plt.bar(x = range(len(prediction)),
        height = prediction)
plt.title('Prediction over countries', size = 12, weight = 'bold')
plt.show()

In [ ]:
most_likely_country = np.argmax(prediction)
country_code = label_encoder.inverse_transform([most_likely_country])
print(f"The most likely country is #{most_likely_country} which corresponds to the country code '{country_code[0]}'")

And that's the end of the inner loop of data science! 

We worked with different data sets, did exploratory analysis, and built & trained a model that can support our song release strategy! 

Now, we get new data every day, right? We need some kind of an automation to train the model with the new data..and that's where the `pipeline` concepts comes in! 🪄✨

head back to the instructions here to continue with the pipelines https://rhoai-mlops.github.io/lab-instructions/#/2-in-the-rhythm-of-data/README